# ⚽️ U-Net 기반 축구 영상 Semantic Segmentation (11 Classes)


## 1. 프로젝트 목적
축구 경기 장면 이미지에서 각 픽셀을 11개 클래스(잔디/선수/골대/관중 등) 중 하나로 분류하는
Semantic Segmentation 모델을 구현하고, 학습 및 평가를 통해 성능과 한계를 분석하는 것이다.

일반 분류(Classification)가 “이미지 1장 = 라벨 1개”라면,
세그멘테이션은 “이미지 1장 = 픽셀 수만큼의 라벨”을 예측해야 한다.  
즉, 같은 모델이라도 난이도가 훨씬 높고, 특히 데이터가 적을 때 과적합이 매우 쉽게 발생한다.


## 2. 데이터
- 데이터셋: Football Semantic Segmentation (UEFA 슈퍼컵 2017 하이라이트 기반)
- 구성: 총 100장 프레임(12프레임 간격 추출 + 일부 대체)
- 클래스 수: 11개
  - Goal Bar, Referee, Advertisement, Ground, Ball, Coaches & Officials, Audience, Goalkeeper A, Goalkeeper B, Team A, Team B


## 3. 문제 설정에서 중요한 점
- 데이터가 매우 적다(100장) → 세그멘테이션에서 과적합이 사실상 기본값처럼 나타날 수 있다.
- 또한 프레임 데이터는 시간적으로 연속성이 있다 → 랜덤 분할은 “비슷한 장면이 train/test에 동시에 들어가는 누수”를 만들 수 있다.

- 그래서 GroupKFold를 사용하여 비슷한 그룹(프레임 번호 기반)이 섞이지 않도록 평가 구조를 잡았다.


## 4. 전체 로드맵
1) 데이터 로드 및 마스크(RGB) → 클래스ID 변환 파이프라인 구축  
2) Baseline U-Net으로 “일단 돌아가는지” 검증(빠른 sanity check)  
3) 전이학습 U-Net(ResNet34 encoder) + feature extraction(encoder freeze) 적용  
4) 손실함수 개선(CE + Dice, 클래스 가중치) + 학습률/규제(weight_decay) 튜닝  
5) 평가 지표는 mIoU 중심으로 기록하고, 결과 이미지는 정성적으로 함께 확인한다


▫️라이브러리 설치 & 임포트

In [ ]:
# segmentation_models_pytorch: 전이학습 U-Net을 빠르게 쓰기 위해 사용
# albumentations: 이미지 증강을 "세그멘테이션용(mask 함께 변환)"으로 안전하게 쓰기 위해 사용

!pip -q install albumentations segmentation-models-pytorch kagglehub

import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import GroupKFold
import segmentation_models_pytorch as smp

# 재현성을 조금이라도 높이기 위한 시드 고정
def seed_everything(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


▫️데이터 다운로드 + 파일 매칭(원본 jpg ↔ fuse 마스크)

In [ ]:
import kagglehub

# KaggleHub로 데이터 다운로드
path = kagglehub.dataset_download("sadhliroomyprime/football-semantic-segmentation")
print("dataset path:", path)

image_folder = os.path.join(path, "images")
file_list = os.listdir(image_folder)

# 원본(.jpg)과 fuse 마스크 파일을 구분
original_files = sorted([f for f in file_list if f.endswith(".jpg") and "fuse" not in f])
fuse_files = sorted([f for f in file_list if "fuse" in f])

# 원본 파일명에서 base_name을 추출해 fuse와 매칭
image_pairs = []
mask_files = []
for orig_file in original_files:
    base_name = orig_file.replace(".jpg", "")
    fuse_file = next((f for f in fuse_files if base_name in f), None)
    if fuse_file:
        image_pairs.append((orig_file, fuse_file))
        mask_files.append(fuse_file)

print("pairs:", len(image_pairs))  # 100 pairs


▫️데이터 샘플 확인(원본/마스크 5쌍)

In [ ]:
# 마스크가 제대로 붙어있는지 초반에 눈으로 확인하는 게 세그멘테이션에선 필수
num_samples = 5
sample_pairs = image_pairs[:num_samples]

fig, axes = plt.subplots(num_samples, 2, figsize=(10, 4*num_samples))

for i, (img_name, mask_name) in enumerate(sample_pairs):
    img = cv2.cvtColor(cv2.imread(os.path.join(image_folder, img_name)), cv2.COLOR_BGR2RGB)
    mask = cv2.cvtColor(cv2.imread(os.path.join(image_folder, mask_name)), cv2.COLOR_BGR2RGB)

    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f"Image: {img_name}")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(mask)
    axes[i, 1].set_title(f"Mask: {mask_name}")
    axes[i, 1].axis("off")

plt.tight_layout()
plt.show()


▫️마스크(RGB) → 클래스ID 매핑 만들기 + 클래스 픽셀 수 집계

In [ ]:
# - 정답 마스크는 "RGB 색상"으로 클래스를 표현한다.
# - 모델 학습을 위해서는 RGB가 아니라 "클래스 번호(0~10)"로 바꿔야 한다.
# - 추가로 클래스별 픽셀 수를 세면, 클래스 불균형 대응(가중치 loss)에 활용 가능하다.

def collect_unique_colors(image_folder, mask_files):
    color_set = set()
    for mf in mask_files:
        mask = cv2.cvtColor(cv2.imread(os.path.join(image_folder, mf)), cv2.COLOR_BGR2RGB)
        uniq = np.unique(mask.reshape(-1, 3), axis=0)
        for c in uniq:
            color_set.add(tuple(c))
    return sorted(list(color_set))

unique_colors = collect_unique_colors(image_folder, mask_files)
print("unique colors:", len(unique_colors))

# 색상 -> 라벨 id (0~10)
color_to_label = {color: idx for idx, color in enumerate(unique_colors)}
label_to_color = {v: k for k, v in color_to_label.items()}

# 클래스별 픽셀 수 계산(가중치 산출용)
color_counts = {color: 0 for color in unique_colors}
total_pixels = 0

for mf in mask_files:
    mask = cv2.cvtColor(cv2.imread(os.path.join(image_folder, mf)), cv2.COLOR_BGR2RGB)
    flat = mask.reshape(-1, 3)
    total_pixels += flat.shape[0]
    uniq, counts = np.unique(flat, axis=0, return_counts=True)
    for u, cnt in zip(uniq, counts):
        u = tuple(u)
        if u in color_counts:
            color_counts[u] += int(cnt)

# 클래스 가중치(단순 역빈도 형태)
class_pixel_counts = np.array([color_counts[label_to_color[i]] for i in range(len(unique_colors))])
class_weights = total_pixels / (len(unique_colors) * class_pixel_counts)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

print("class_weights_tensor shape:", class_weights_tensor.shape)


▫️ Dataset 정의(Albumentations 포함) + Transform 정의

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_transforms(is_train=True):
    # - 세그멘테이션 증강은 image와 mask를 "같이" 변환해야 한다.
    # - 너무 과한 증강은 작은 데이터에서 오히려 해가 될 수 있어, 기본적인 수준으로만 구성.
    if is_train:
        return A.Compose([
            A.Resize(256, 256),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.3),
            A.Normalize(),
            ToTensorV2(),
        ])
    else:
        return A.Compose([
            A.Resize(256, 256),
            A.Normalize(),
            ToTensorV2(),
        ])

class FootballDataset(Dataset):
    def __init__(self, image_files, mask_files, image_folder, color_to_label, transform=None):
        self.image_files = image_files
        self.mask_files = mask_files
        self.image_folder = image_folder
        self.color_to_label = color_to_label
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_folder, self.image_files[idx])
        mask_path = os.path.join(self.image_folder, self.mask_files[idx])

        image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        mask_rgb = cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2RGB)

        # RGB 마스크를 클래스ID 마스크로 변환
        mask_class = np.zeros(mask_rgb.shape[:2], dtype=np.uint8)
        for color, label in self.color_to_label.items():
            mask_class[(mask_rgb == color).all(axis=-1)] = label

        if self.transform:
            out = self.transform(image=image, mask=mask_class)
            image = out["image"]         # torch tensor (C,H,W)
            mask_class = out["mask"]     # numpy -> torch tensor로 변환 필요

        return image, torch.tensor(mask_class, dtype=torch.long)


▫️ Baseline - 일단 돌아가는지 체크
- 세그멘테이션은 파이프라인이 한 군데만 틀어져도 학습이 전부 무너진다. 그래서 먼저 간단한 설정으로 “학습/예측/시각화가 정상 동작하는지”를 확인한 뒤, 전이학습 + 교차검증으로 넘어감.

◻️ Main 모델(전이학습 U-Net) 정의: ResNet34 encoder + freeze

In [ ]:
# - 작은 데이터(100장)에서는 전체를 다 학습시키면 과적합이 너무 빨리 온다.
# - 그래서 encoder는 ImageNet으로 학습된 특징을 "고정"하고,
#   decoder 중심으로 학습(feature extraction)해서 안정성을 높인다.

def get_model():
    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=11
    )
    # encoder freeze
    for p in model.encoder.parameters():
        p.requires_grad = False
    return model


▫️ Loss(Dice) + CombinedLoss(CE+Dice) + mIoU 함수

In [ ]:
# Dice Loss: 겹침(유사도)을 직접 최적화하려는 목적 - 세그멘테이션에 자주 사용
def dice_loss(pred, target, num_classes=11, smooth=1e-7):
    # pred: (B, C, H, W) logits
    pred_soft = F.softmax(pred, dim=1)

    # target: (B, H, W) -> one-hot: (B, C, H, W)
    target_oh = F.one_hot(target, num_classes=num_classes).permute(0, 3, 1, 2).float()

    intersection = torch.sum(pred_soft * target_oh, dim=(2, 3))
    union = torch.sum(pred_soft, dim=(2, 3)) + torch.sum(target_oh, dim=(2, 3))
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1.0 - dice.mean()

class CombinedLoss(nn.Module):
    # - CE는 픽셀 단위 분류를 잘 잡아주고,
    # - Dice는 "영역 겹침"을 밀어주는 성격이라 둘을 섞으면 성능이 안정되는 경우가 많다.
    def __init__(self, alpha=0.3, weight=None):
        super().__init__()
        self.alpha = alpha
        self.ce = nn.CrossEntropyLoss(weight=weight)
    def forward(self, pred, target):
        return self.alpha * self.ce(pred, target) + (1 - self.alpha) * dice_loss(pred, target)

@torch.no_grad()
def calculate_miou(pred, target, num_classes=11):
    pred_cls = torch.argmax(pred, dim=1)  # (B,H,W)
    ious = []
    for cls in range(num_classes):
        pred_inds = (pred_cls == cls)
        tgt_inds = (target == cls)
        intersection = (pred_inds & tgt_inds).sum().item()
        union = (pred_inds | tgt_inds).sum().item()
        if union == 0:
            ious.append(np.nan)
        else:
            ious.append(intersection / union)
    return float(np.nanmean(ious))


▫️GroupKFold용 그룹 생성(프레임 번호 기반)

In [ ]:
# - 프레임 데이터는 서로 매우 비슷한 장면이 연속으로 등장한다.
# - 랜덤 split이면 "비슷한 장면이 train/val에 동시에 들어가는 누수"가 생길 수 있다.
# - 그래서 파일명에 들어있는 숫자(프레임 번호)를 그룹으로 묶어 GroupKFold를 구성한다.

def extract_number(s):
    m = re.findall(r"\d+", s)
    return int(m[0]) if m else 0

groups = np.array([extract_number(f) for f in original_files])
gkf = GroupKFold(n_splits=5)

print("example groups:", groups[:10])


▫️1 Fold 학습 함수(run_fold) + Warmup/Poly LR + EarlyStopping

In [ ]:
import copy

def warmup_poly_lr_lambda(epoch, warmup_epochs=5, max_epochs=50, power=0.9):
    # - Warmup: 초반 학습 불안정 완화
    # - Poly decay: segmentation에서 자주 쓰는 안정적인 decay 형태
    if epoch < warmup_epochs:
        return (epoch + 1) / warmup_epochs
    else:
        t = (epoch - warmup_epochs) / max(1, (max_epochs - warmup_epochs))
        return (1 - t) ** power

def run_fold(fold, model, train_ds, val_ds, class_weights_tensor, device, max_epochs=50, patience=10):
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)

    criterion = CombinedLoss(alpha=0.3, weight=class_weights_tensor)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
    scheduler = torch.optim.lr_scheduler.LambdaLR(
        optimizer,
        lr_lambda=lambda e: warmup_poly_lr_lambda(e, warmup_epochs=5, max_epochs=max_epochs)
    )

    best_val_miou = -1
    best_wts = copy.deepcopy(model.state_dict())
    patience_counter = 0

    for epoch in range(max_epochs):
        # ---- train ----
        model.train()
        train_loss = 0.0
        for imgs, masks in train_loader:
            imgs, masks = imgs.to(device), masks.to(device)

            optimizer.zero_grad()
            logits = model(imgs)               # (B, C, H, W)
            loss = criterion(logits, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= max(1, len(train_loader))

        # ---- val ----
        model.eval()
        val_loss = 0.0
        val_miou = []
        with torch.no_grad():
            for imgs, masks in val_loader:
                imgs, masks = imgs.to(device), masks.to(device)
                logits = model(imgs)
                val_loss += criterion(logits, masks).item()
                val_miou.append(calculate_miou(logits, masks, num_classes=11))

        val_loss /= max(1, len(val_loader))
        avg_val_miou = float(np.mean(val_miou))
        current_lr = optimizer.param_groups[0]["lr"]

        # 로그(네 노트북 스타일 반영)
        msg = f"Epoch [{epoch+1}/{max_epochs}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val mIoU: {avg_val_miou:.4f}, LR: {current_lr:.6f}"
        if avg_val_miou > best_val_miou:
            best_val_miou = avg_val_miou
            best_wts = copy.deepcopy(model.state_dict())
            patience_counter = 0
            msg += " = BEST"
        else:
            patience_counter += 1

        print(msg)
        scheduler.step()

        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    print(f"Fold {fold+1} 완료 Best Val mIoU: {best_val_miou:.4f}\n")
    return best_val_miou, best_wts


▫️ K-Fold 실행 + 최고 Fold 모델 저장 + 전체 mIoU 평가

In [ ]:
fold_best = []
fold_wts = []

for fold, (tr_idx, va_idx) in enumerate(gkf.split(original_files, groups=groups)):
    train_imgs = [original_files[i] for i in tr_idx]
    train_masks = [mask_files[i] for i in tr_idx]
    val_imgs = [original_files[i] for i in va_idx]
    val_masks = [mask_files[i] for i in va_idx]

    train_ds = FootballDataset(train_imgs, train_masks, image_folder, color_to_label, transform=get_transforms(True))
    val_ds   = FootballDataset(val_imgs,   val_masks,   image_folder, color_to_label, transform=get_transforms(False))

    model = get_model().to(device)
    best_miou, best_wts = run_fold(fold, model, train_ds, val_ds, class_weights_tensor, device)

    fold_best.append(best_miou)
    fold_wts.append(best_wts)

best_fold = int(np.argmax(fold_best))
print("best_fold:", best_fold+1, "best_val_miou:", fold_best[best_fold])

# best fold weights 로드
best_model = get_model().to(device)
best_model.load_state_dict(fold_wts[best_fold])
best_model.eval()

# 전체 데이터 mIoU 평가(네 노트북 로그: Final mIoU on the entire dataset: 0.4615)
full_ds = FootballDataset(original_files, mask_files, image_folder, color_to_label, transform=get_transforms(False))
full_loader = DataLoader(full_ds, batch_size=4, shuffle=False, num_workers=0)

miou_list = []
with torch.no_grad():
    for imgs, masks in full_loader:
        imgs, masks = imgs.to(device), masks.to(device)
        logits = best_model(imgs)
        miou_list.append(calculate_miou(logits, masks, num_classes=11))

final_miou = float(np.mean(miou_list))
print(f"Final mIoU on the entire dataset: {final_miou:.4f}")

'''
Fold Best Val mIoU 예시: 0.5359
Final mIoU: 0.4615
'''

▫️시각화(원본 / GT / Pred) + 색상 입히기
- 점수만 보면 “좋아진 것처럼” 보여도, 실제 분할이 이상하게 깨지는 경우가 있어 세그멘테이션은 시각화가 필수다.

In [ ]:
def mask_to_rgb(mask_2d, label_to_color):
    h, w = mask_2d.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)
    for lbl, color in label_to_color.items():
        rgb[mask_2d == lbl] = np.array(color, dtype=np.uint8)
    return rgb

# 몇 장만 확인
indices = [0, 10, 20, 30, 40]
fig, axes = plt.subplots(len(indices), 3, figsize=(12, 4*len(indices)))

best_model.eval()
with torch.no_grad():
    for r, idx in enumerate(indices):
        img_t, gt = full_ds[idx]
        logits = best_model(img_t.unsqueeze(0).to(device))
        pred = torch.argmax(logits, dim=1).squeeze(0).cpu().numpy()

        # albumentations normalize된 이미지를 화면용으로 되돌리기(대략적)
        img_np = img_t.permute(1,2,0).cpu().numpy()
        img_np = (img_np - img_np.min()) / (img_np.max() - img_np.min() + 1e-8)

        gt_rgb = mask_to_rgb(gt.cpu().numpy(), label_to_color)
        pred_rgb = mask_to_rgb(pred, label_to_color)

        axes[r, 0].imshow(img_np); axes[r, 0].set_title("Image"); axes[r, 0].axis("off")
        axes[r, 1].imshow(gt_rgb); axes[r, 1].set_title("GT Mask"); axes[r, 1].axis("off")
        axes[r, 2].imshow(pred_rgb); axes[r, 2].set_title("Pred Mask"); axes[r, 2].axis("off")

plt.tight_layout()
plt.show()


# 🟨 최종 결론 및 분석

---

## 1. 결과 요약
본 실험에서는 축구 경기 장면(100장)에서 11개 클래스를 픽셀 단위로 분할하는 U-Net 기반 세그멘테이션 모델을 구축했다.
데이터가 매우 적고(100장), 프레임 특성상 시간적 유사성이 강해 데이터 누수 가능성이 높기 때문에,
평가 구조는 GroupKFold로 설계했다.

- 교차검증 중 최고 성능(Best Val mIoU): 0.5359
- 전체 데이터 기준 최종 평가(Final mIoU): 0.4615

---

## 2. 관찰된 현상
1) 과적합이 쉽게 발생한다
- 데이터 수가 적고 클래스는 11개로 많아, 모델이 “일반적인 규칙”보다 “특정 장면”을 외우기 쉬웠다.
- 실제로 학습이 진행되며 train 지표는 좋아지는데, val 성능이 불안정하거나 정성 결과가 오히려 망가지는 패턴을 확인했다.

2) 이 미션에서는 partial fine-tuning보다 feature extraction이 더 안정적이었다
- 인코더(ResNet34)를 freeze하고 디코더 중심으로 학습하는 방식이 더 높은 성능을 보였다.
- 작은 데이터에서는 학습 자유도가 큰 방식(partial FT)이 오히려 과적합을 더 빠르게 만든다고 해석할 수 있다.

3) 손실함수와 규제/학습률이 성능에 민감했다
- CE 단독보다 CE+Dice 조합이 val mIoU 개선에 도움을 주었다.
- 또한 learning rate와 weight_decay 조정만으로도 성능이 유의미하게 흔들렸다.
  즉, 이 문제는 “모델 바꾸기”보다 “학습 안정화”의 영향이 큰 환경이었다.

---

## 3. 한계
- 데이터가 100장으로 지나치게 적어, 정량 지표(mIoU)가 특정 분할 방식에 크게 의존할 수 있다.
- RGB 마스크 색상 → 클래스 ID 매핑이 데이터 로딩 단계에서 결정되므로, 매핑을 고정/검증하지 않으면 재현성이 흔들릴 수 있다.
- 결과 이미지 기준으로는 “점수는 괜찮아도 시각적으로는 더 엉망”인 사례가 존재했는데,
  이는 작은 데이터에서 지표 최적화가 곧바로 정성 품질로 이어지지 않을 수 있음을 보여준다.

---

## 4. 다음 개선 방향
- 데이터 증강을 “과하게” 넣기보다, 실제 경기 환경 변화(밝기/색상/좌우반전 등)에 맞게 제한적으로 적용
- 클래스 불균형이 심한 클래스(공, 골대 등)에 대해 가중치/샘플링 전략을 추가 검토
- 조건이 허용된다면 프레임 수를 늘리거나(더 많은 영상 프레임), 동일 경기 외 다른 경기로 도메인을 확장하여 일반화 성능 확인

---

## 5. 결론
소규모 축구 프레임 데이터에서 U-Net 세그멘테이션을 구현하고, 데이터 누수 방지(GroupKFold), 전이학습(feature extraction), 손실함수(CE+Dice) 및 학습 안정화(규제/스케줄)를 통해
mIoU 기준 성능 개선을 확인했다.
다만 데이터 한계로 과적합이 강하게 나타나므로, 이후 실험은 “일반화”를 목표로 데이터/평가 구조의 보강이 필요하다.
